# Compute!'s Apple II Proofreader (Deconstructed and Re-Created)

## Apple Automatic Proofreader from Compute! Magazine
(This cell from: [Hartze11's A2Prooferader github](https://github.com/hartze11/a2_proofreader/tree/master):)

From Wikipedia [1]:

The Automatic Proofreader is a series of checksum utilities published by COMPUTE! Publications for its COMPUTE! and COMPUTE!'s Gazette magazines, and various books. These programs are designed to allow home computer users to easily detect errors on BASIcktype-in programs, and work by displaying a hash value for each line entered that can be compared against the reference value printed in the magazine. Initially published for use with the Commodore 64 and VIC-20 in 1983, the Proofreader was later made available for the Atari 8-bit family, Apple II family,[2] and IBM PC/PCjr as well.

The line-by-line "real-time" feedback feature was something of a novelty at the time and represented a significant improvement over earlier checksum utilities, which were typically run only after a user program had been entered and, due to quite simplistic checksum algorithms, had trouble catching errors like transposed characters.

[1] https://en.wikipedia.org/wiki/The_Automatic_Proofreader [2] https://archive.org/stream/1985-07-compute-magazine/Compute_Issue_062_1985_Jul#page/n75/mode/2up



Here is the original Compute! Apple II Proofreader code:

(This listing from: [Hartze11's A2Prooferader github](https://github.com/hartze11/a2_proofreader/tree/master): )

```basic
10 C = 0: FOR I = 768 TO 768 + 68: READ A: C = C + A: POKE I,A: NEXT 
20 IF C < > 7258 THEN PRINT "ERROR IN PROOFREADER DATA STATEMENTS": END 
30 IF PEEK (190 * 256) < > 76 THEN POKE 56,0: POKE 57,3: CALL 1002: GOTO 50 
40 PRINT CHR$ (4); "IN#A$300" 
50 POKE 34,0: HOME : POKE 34,1: VTAB 2: PRINT "PROOFREADER INSTALLED" 
100 DATA 216,32,27,253,201,141 
110 DATA 208,60,138,72,169,0 
120 DATA 72,189,255,1,201,160 
130 DATA 240,8,104,10,125,255 
140 DATA 1,105,0,72,202,208 
150 DATA 238,104,170,41,15,9 
160 DATA 48,201,58,144,2,233 
170 DATA 57,141,1,4,138,74 
180 DATA 74,74,74,41,15,9 
190 DATA 48,201,58,144,2,233 
200 DATA 57,141,0,4,104,170 
210 DATA 169,141,96
```

The code `POKE`s the bytes in the `DATA` statements into memory address 768 (`$0300`).  It keeps a running tally of the values, and if the tally doesn't match the expected value, then there was a `DATA` entry issue and it stops.  Then it checks the type of Apple the program is being run on, and executes the binary program in one of two ways, by either `CALL 1002` or running the DOS command `IN#A$300` (which initializes address $0300). Then it changes the text-scrolling window so the top line doesn't get overwritten if the screen scrolls down.

Hartze11 actually captured the code and the assembly for it:

(This listing from: [source](https://github.com/hartze11/a2_proofreader/blob/master/proofreader.asm#L1) )

```assembler
0300-   D8          CLD
0301-   20 1B FD    JSR   $FD1B
0304-   C9 8D       CMP   #$8D
0306-   D0 3C       BNE   $0344
0308-   8A          TXA
0309-   48          PHA
030A-   A9 00       LDA   #$00
030C-   48          PHA
030D-   BD FF 01    LDA   $01FF,X
0310-   C9 A0       CMP   #$A0
0312-   F0 08       BEQ   $031C
0314-   68          PLA
0315-   0A          ASL
0316-   7D FF 01    ADC  $01FF,X
0319-   69 00       ADC  #$00
031B-   48          PHA
031C-   CA          DEX
031D-   D0 EE       BNE   $030D
031F-   68          PLA
0320-   AA          TAX
0321-   29 0F       AND   #$0F
0323-   09 30       ORA   #$30
0325-   C9 3A       CMP   #$3A
0327-   90 02       BCC   $032B
0329-   E9 39       SBC   #$39
032B-   8D 01 04    STA   $0401
032E-   8A          TXA
032F-   4A          LSR
0330-   4A          LSR
0331-   4A          LSR
0332-   4A          LSR
0333-   29 0F       AND   #$0F
0335-   09 30       ORA   #$30
0337-   C9 3A       CMP   #$3A
0339-   90 02       BCC   $033D
033B-   E9 39       SBC   #$39
033D-   8D 00 04    STA   $0400
0340-   68          PLA
0341-   AA          TAX
0342-   A9 8D       LDA   #$8D
0344-   60          RTS
```

This was the code from [AppleWin](https://github.com/AppleWin/AppleWin)'s disassembler that I personally captured:

```asm
0300:D8                   CLD                     
0301:20 1B FD             JSR KEYIN               
0304:C9 8D                CMP #$8D         #-115 M
0306:D0 3C                BNE $0344      0344     
0308:8A                   TXA                     
0309:48                   PHA                     
030A:A9 00                LDA #$00               @
030C:48                   PHA                     
030D:BD FF 01             LDA INPUT.B-1,X01FF:37 7
0310:C9 A0                CMP #$A0          #-96  
0312:F0 08                BEQ $031C     031C  
0314:68                   PLA                     
0315:0A                   ASL                     
0316:7D FF 01             ADC INPUT.B-1,X01FF:37 7
0319:69 00                ADC #$00               @
031B:48                   PHA                     
031C:CA                   DEX                     
031D:D0 EE                BNE $030D      030D     
031F:68                   PLA                     
0320:AA                   TAX                     
0321:29 0F                AND #$0F          #+15 O
0323:09 30                ORA #$30          #+48 0
0325:C9 3A                CMP #$3A          #+58 :
0327:90 02                BCC $032B      032B     
0329:E9 39                SBC #$39          #+57 9
032B:8D 01 04             STA LINE1+1    0401:A0  
032E:8A                   TXA                     
032F:4A                   LSR                     
0330:4A                   LSR                     
0331:4A                   LSR                     
0332:4A                   LSR                     
0333:29 0F                AND #$0F          #+15 O
0335:09 30                ORA #$30          #+48 0
0337:C9 3A                CMP #$3A          #+58 :
0339:90 02                BCC $033D      033D     
033B:E9 39                SBC #$39          #+57 9
033D:8D 00 04             STA LINE1      0400:A0  
0340:68                   PLA                     
0341:AA                   TAX                     
0342:A9 8D                LDA #$8D         #-115 M
0344:60                   RTS                     
0345:00                   BRK                     
0346:00                   BRK                     
```


## Breakdown of Assembly steps
1. Apple II computers take ASCII input, but the keyboard has the high-bit set. X becomes the character count + 1 (for the <CR>)
2. The GETLN program takes input until enter is pressed.  It returns the input buffer exactly as it was typed, in the `$0200` block of memory.  
3. The register X is the length of the string + 1. 
4. The program breaks down as follows:  
5. Wait for Enter (`$0300`-`$0306`)
6. Setup the Accumulator (push 0 onto it) 
7. Load the character that is in the buffer at `$01FF + X` (AKA starting from the last letter - it uses `$01FF` instead of `$0200` to ignore the <CR> ). 
8. If character is a space, skip it and decrement X.
9. Pop Stack into Accumulator  (get last turn's result)
10. Shift Accumulator left (multiply by 2, the carry bit gets set with what got shifted out if the result is > 255)
11. Add-with-carry the Accumulator with the the current character in the buffer. (accumulator + buffer character + carry)
12. Add-with-carry 0 (accmulator + 0 + carry, aka accumulator + carry)
13. Push Accumulator onto Stack
14. Decrement X
15. Continue from step 7, backwards through the input, until x is 0
16. Output the accumulator result in human-readable hexadecimal to the top of the screen.

In [12]:
# Here's the checksum-generator + test-cases in Python:

def checksum_orig(input_string):
    """
    Processes a string and returns the "Compute! Apple II Proofreader" checksum for that string.
    """
    
    def carr(num):
        return (num > 0xFF) 
    
    def chop(num):
        return (num & 0xFF)

    def op_LSR(num, n=1):
        val = num << n
        return carr(val), chop(val)

    def op_ADC(num, val, carry):
        val = num + val + carry
        return carr(val), chop(val)

    #turn string to high-bit chars
    input_chars = []
    for char in input_string:
        input_chars.append(ord(char) + 0x80)  

    car = 0  # Emulate the carry flag
    acc = 0  # Emulating the accumulator register
    for char in reversed(input_chars):
        if char == 0xA0:  # 
            continue  # Skip spaces (0x20 is 0xA0 in high-bit-ASCII)
        #1. lsr, 2. acc+char+carry, 3. acc+carry
        car, acc = op_LSR(acc) 
        car, acc = op_ADC(acc, char, car)
        acc += car  #AKA car, acc= op_ADC(acc, 0, car)   
    return acc

def hex(s):
    return f"{s:02X}"            

# Define test cases
test_cases = [
    ["Print Tests:","",None],
    ["10 PRINT", "98",True],
    ["10?", "98",False],
    ["10?", "12",True],
    ["390 PRINT", "F4",True],
    ["390PRINT", "F4",True], 
    ["Before and After Tests:","",None],
    ["180 C= CCBD(I)):P = P(BDU)) :D = D(BD(I)): RETURN", "A3",False], 
    ["180 C= C(BD(I)):P = P(BD(I)):D = D(BD(I)): RETURN", "A3",True], 
    ["MLX Checksum Calc Code:", "", None],
    ["560 C= INT (B/256): C= B - 254*C- 255*(C>127): C= C- 255*(C>255)", "28",True],
    ["570 FOR F=1 TO 8: C= C*2 - 255*(C>127) + V(F): C= C- 255*(C>255): NEXT: RETURN","20",True],
    ["REM Tests:","",None],
    ["10 REM * COPYRIGHT 1987 *", "B8",True],
    ["20 REM * COPYRIGHT 1987 *", "B8",False],    
    ["10REM* COPYRIGHT 1987 *", "B8",True],
    ["10REM*COPYRIGHT1987*", "B8",True],
    ["10REM*COPYRIGHT1978*", "B8",False],
]

# Test framework
algs = ["checksum_orig"]
for alg in algs:
    print ("___"+alg+"___")
    for test in test_cases:
        input_data, expected_checksum, is_pass = test
        if (expected_checksum == ""):
            print("-" * 60)
            print(input_data)
            continue  
        computed_checksum = hex(globals()[alg](input_data))    
        print("✅" if (computed_checksum == expected_checksum) else "❌", f"{expected_checksum}/{computed_checksum} `{input_data}`" )
    print("-" * 60)


___checksum_orig___
------------------------------------------------------------
Print Tests:
✅ 98/98 `10 PRINT`
❌ 98/12 `10?`
✅ 12/12 `10?`
✅ F4/F4 `390 PRINT`
✅ F4/F4 `390PRINT`
------------------------------------------------------------
REM Tests:
✅ B9/B9 `20 REM * COPYRIGHT 1987 *`
❌ B9/B8 `10 REM * COPYRIGHT 1987 *`
✅ B9/B9 `20 REM*COPYRIGHT 1987*`
❌ B9/BB `20 REM * COPYRIGHT 1978 *`
------------------------------------------------------------
Before and After Tests:
❌ A3/77 `180 C= CCBD(I)):P = P(BDU)) :D = D(BD(I)): RETURN`
✅ A3/A3 `180 C= C(BD(I)):P = P(BD(I)):D = D(BD(I)): RETURN`
------------------------------------------------------------
MLX Checksum Calc Code:
✅ 28/28 `560 C= INT (B/256): C= B - 254*C- 255*(C>127): C= C- 255*(C>255)`
✅ 20/20 `570 FOR F=1 TO 8: C= C*2 - 255*(C>127) + V(F): C= C- 255*(C>255): NEXT: RETURN`
------------------------------------------------------------


# Apple MLX Checksum Calculator

_Compute!_ realized that some code had to be in machine language for speed or capability, so they had to create an easy way for readers to type it in. 

Typing in Machine Language (assembly or ML) code was difficult to do by hand.

This checksum is a bit easier, as the formula is in BASIC.  
Here's a breakdown of the code, with the subroutines divided out.  
The checksum calculation is performed by subroutine 560-570.
It calculates the checksum by line number and 8 bytes of data, with the ninth value being the checksum.

This Basic program is fun because it creates its own flashing replacement cursor, captures the input, and filters keypresses to 0-F when in Machine Language entry mode. 
Also, you can see the A2Proofreader checksums in half-size font next to the line numbers - We're using the previous verifier program to type in another verifier program - Nice!

Also, it turns out the that the C64/C128 MLX calculation formulas are the exact same as the Apple MLX formula, so we get two proofreader checksums for the price of one. 

![](./MLX%20deconstructed-2.png)

In [6]:
def hex(s):
    return f"{s:02X}"

def find_mlx_checksum_original(b, v):
    # b is beginning address in decimal,
    # v is 8 bytes in decimal
    # returns checksum as a byte c
    c = b // 256   # (ASL b, 8) (b >> 8) 
    c = b - 254*c - 255*(c>127)
    c =         c - 255*(c>255)
    for f in v:  
        c =   2*c - 255*(c>127) + f
        c =     c - 255*(c>255)
    return c

def find_mlxc64_checksum_v2(ad, aa):
    """ Same as Apple II checksum"""
    """ Commodore BASIC returns 0 for false and -1 for true (true story!)"""
    # ad is beginning address in decimal,
    # aa is 8 bytes in decimal
    # returns checksum as a byte ck
    ck= ad // 256   # (ad >> 8) 
    ck= ad - 254*ck + 255*(ck>127)*-1 
    ck= ck + 255*(ck>255)*-1
    for a in aa:  
        ck= ck * 2 + 255*(ck>127)*-1 + a
        ck= ck + 255*(ck>255)*-1
    return ck & 0xFF # chop off overflow 

# a shortened version by mathemetician Joseph Nebus 
# [https://nebusresearch.wordpress.com/2021/06/21/heres-some-matlab-octave-code-for-your-mlx-simulator/]
# adapted to python by yours truly
def mlxIIslick(address,entries):
    ck = (address - 254*(address // 256)) % 255 
    ck += sum(e * (2**i) for e, i in zip(entries, range(7, -1, -1)))
    ck = ck % 255
    ck += (255 if ck == 0 else 0 )
    return ck

def find_mlx_checksum_short(b, v):
    """ Not really shorter, just uses shifts instead of """ 
    # b is beginning address in decimal,
    # v is 8 bytes in decimal
    # returns checksum as a byte c
    c = b >> 8
    c = b - 254*c - 255*(c>127)
    c =         c - 255*(c>255)       
    for f in v:  
        c = (c<<1) - 255*(c>127) + f
        c =      c - 255*(c>255)       
    return c

def find_mlx_checksum(b,v, func=mlxIIslick):
    return func(b,v)
    
def process_string(s, func=find_mlx_checksum_original):
    """ condition the strings """
    s = s.replace(" ","")
    s = s.replace(":","")
    #print (f"{s} L:{len(s)}")
    if (len(s) != 22):
        return
    s = s[0:4]+" "+s[4:6]+" "+s[6:8]+" "+s[8:10]+" "+s[10:12]+" "+s[12:14]+" "+s[14:16]+" "+s[16:18]+" "+s[18:20]+" "+s[20:22]
    return s

def code_to_vals(s):
    a = s.split()    
    int_list = [int(h,16) for h in a]
    return int_list

def process_test(s, func=find_mlx_checksum_original):
    """ split out address and values and run func """
    #print("  ",int_list)
    int_list = code_to_vals(s)
    c = func(int_list[0], int_list[1:-1])
    return (hex(c))

#samples: 
test_cases = [
"*A2 ML formatting variations",
"6000 D8 78 85 45 86 46 84 47 ED", #A2 ML formatting variations  
"6008 A6 07 0A 0A B0 04 10 3E B3", #A2 ML
"6008: A6 07 0A 0A B0 04 10 3E B3", #A2 ML
"6008A6070A0AB004103EB3",
"6008:A6070A0AB004103EB3",
"*Edge Cases",
"0848: 20 A9 00 8D 53 1E A0 00 FF", ## FF edge case
"0FE9: 20 D0 0A A9 2E 99 FF 01 01", ## 01 edge case 
"*00/FF substitutions - Broken Edge case",
"0848: 20 A9 00 8D 53 1E A0 FF FF", ## FF edge case (00 / FF) substituted - will not catch this
"*C64 ML",
"0801: 0B 08 0A 00 9E 32 30 36 2E", #C64 ML
"0809: 31 00 00 00 A9 00 8D 21 3B", #C64 ML
"*Bad entries",
"6008:A6071A0AB004103EB3",   #bad entry 
"6010:A6070A0AB004103EB3",   #wrong address    
"6008:A6070A0AB004103B3",    #too few chars
"6008:A6070A0AB004103EEB3",  #too many chars
]

def FF00_score(s):
    vals = s.split()[1:-1]
    return vals.count("FF") + vals.count("00") 

def batch_run_tests(func):
    print("===",str(func)[1:-23],"===")
    for test in test_cases:
        if (test[:1] == "*"):
            print("-" * 60)
            print(test)
            continue  
        test_input= process_string(test)
        if not test_input: 
            print (f"bad input {test_input}")
            continue            
        expected_checksum = test_input[-2:]
        computed_checksum = process_test(test_input, func)
        F0 = FF00_score(test_input)
        print(f"`{test_input}`","✅" if computed_checksum == expected_checksum else f"❌ {expected_checksum}/{computed_checksum}", f"⚠" if F0 > 0 else "")
    print("-" * 60)
    #print("⚠ = Lines with 00 or FF; MLX does not detect FF/00 substitutions (unlikely in practice)" )

batch_run_tests(find_mlx_checksum_original)
batch_run_tests(find_mlxc64_checksum_v2)
batch_run_tests(mlxIIslick)



=== function find_mlx_checksum_original ===
------------------------------------------------------------
*A2 ML formatting variations
`6000 D8 78 85 45 86 46 84 47 ED` ✅ 
`6008 A6 07 0A 0A B0 04 10 3E B3` ✅ 
`6008 A6 07 0A 0A B0 04 10 3E B3` ✅ 
`6008 A6 07 0A 0A B0 04 10 3E B3` ✅ 
`6008 A6 07 0A 0A B0 04 10 3E B3` ✅ 
------------------------------------------------------------
*Edge Cases
`0848 20 A9 00 8D 53 1E A0 00 FF` ✅ ⚠
`0FE9 20 D0 0A A9 2E 99 FF 01 01` ✅ ⚠
------------------------------------------------------------
*00/FF substitutions
`0848 20 A9 00 8D 53 1E A0 FF FF` ✅ ⚠
------------------------------------------------------------
*C64 ML
`0801 0B 08 0A 00 9E 32 30 36 2E` ✅ ⚠
`0809 31 00 00 00 A9 00 8D 21 3B` ✅ ⚠
------------------------------------------------------------
*Bad entries
`6008 A6 07 1A 0A B0 04 10 3E B3` ❌ B3/B5 
`6010 A6 07 0A 0A B0 04 10 3E B3` ❌ B3/BB 
bad input None
bad input None
------------------------------------------------------------
=== function fin

In [ ]:
Ideas for this:

Useful for reading type-in lines from scanned items
make a lister program the uses the checksum routine and puts the checksum next to each line number in inverse.
 - Either an ML program that takes the listings output and intercepts it
 - dump into a text file and then add the checksums in the text file for batch comparison of already-typed in files. 

Add additional checksum programs for different computers (commodore v1 v2 v3, atari, PC), and different magazines.
